In [1]:
import sys
sys.path.append('../')
import torch
from torch import nn
from models.resnet import ResNet18
from models.swin_transformer import Swin_transformer

# external
from experiments.activation.acts import *
from experiments.util.utils import *



resnet
    batch 128
    lr 0.1
    

In [2]:
# cifar10 , resnet18, SiLU
resnet = ResNet18(10,act=SiLU())
state_dict = torch.load('./checkpoint/ckpt_resnet18_SiLU.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

FileNotFoundError: [Errno 2] No such file or directory: './checkpoint/ckpt_resnet18_SiLU.pth'

In [12]:
# cifar10 , resnet18, Swish
resnet = ResNet18(10,act=nn.ReLU())

print_results(resnet,'./checkpoint/ckpt_resnet18_ReLU.pth')

ResNet
total params : 11173962
ReLU param : {}
best acc: 95.43
best ep : 197


In [10]:
# cifar10 , resnet18, Swish
resnet = ResNet18(100,act=Mish())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_Mish.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

{}
76.72
199


In [5]:
# cifar10 , resnet18, Swish
resnet = ResNet18(10,act=nn.ELU())
state_dict = torch.load('./checkpoint/ckpt_resnet18_ELU.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

92.79
194


In [8]:
# cifar10 , resnet18, ASN
resnet = ResNet18(100,act=ASN())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_ASN.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([5.6681], requires_grad=True)
78.1
190


In [3]:
# cifar10 , resnet18, Swish
resnet = ResNet18(10,act=Swish())
state_dict = torch.load('./checkpoint/ckpt_resnet18_Swish.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([5.9302], requires_grad=True)
95.35
179


In [6]:
resnet = ResNet18(10,act=Mish())
state_dict = torch.load('./checkpoint/ckpt_resnet18_Mish_94.08.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

94.17
182


In [20]:
# cifar10 , resnet18, ASN

resnet = ResNet18(10,act=ASN())
state_dict = torch.load('./checkpoint/ckpt_resnet18_asn_177_95.44.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([-6.7338], requires_grad=True)
95.44
177


In [3]:
# cifar10 , resnet18, SiluT

resnet = ResNet18(10,act=SliuT())
state_dict = torch.load('./checkpoint/ckpt_resnet18_SliuT_95.37.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([-6.6251], requires_grad=True)
95.53
184


In [21]:
# cifar10 , resnet18, sASN

resnet = ResNet18(10,act=sASN())
state_dict = torch.load('./checkpoint/ckpt_resnet18_sasn_177_95.18.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([5.9771], requires_grad=True)
95.27
171


In [22]:
# cifar10 , resnet18, Relu

resnet = ResNet18(10,act=nn.ReLU())
state_dict = torch.load('./checkpoint/ckpt_resnet18_relu_177_94.63.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

94.63
177


In [23]:
# cifar10 , resnet18, gelu

resnet = ResNet18(10,act=nn.GELU())
state_dict = torch.load('./checkpoint/ckpt_resnet18_GELU_199_95.05.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

95.05
199


In [ ]:
# cifar10 , resnet18, Relu

resnet = ResNet18(10,act=nn.ReLU())
state_dict = torch.load('./checkpoint/ckpt_resnet18_relu_177_94.63.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

In [9]:
# cifar100 , resnet18, sASN

resnet = ResNet18(100)
resnet.load_state_dict(torch.load('./checkpoint/ckpt_resnet18_cifar100_sasn_199_79.59.pth')['net'])
print_act_p(resnet)

Parameter containing:
tensor([6.4185], requires_grad=True)


In [12]:
# cifar10 , resnet18, ASN

resnet = ResNet18(10)
resnet.load_state_dict(torch.load('./checkpoint/ckpt_resnet18_asn_177_95.44.pth')['net'])
print_act_p(resnet)

Parameter containing:
tensor([-6.7338], requires_grad=True)


In [10]:
# cifar10 , resnet18, ASN
resnet = ResNet18(100,act=ASN())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_ASN_78.10.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([5.6681], requires_grad=True)
78.1
190


In [5]:
# cifar100 , resnet18, SwishT

resnet = ResNet18(100,act=SwishT())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_sasn_199_79.59.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([6.4185], requires_grad=True)
79.68
194


In [11]:
# cifar10 , resnet18, ASN
resnet = ResNet18(100,act=Mish())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_Mish.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

77.03
190


In [14]:
# cifar10 , resnet18, sASN=swisht

resnet = ResNet18(10)
resnet.load_state_dict(torch.load('./checkpoint/ckpt_resnet18_sasn_177_95.18.pth')['net'])
print_act_p(resnet)

Parameter containing:
tensor([5.9771], requires_grad=True)


In [1]:
# cifar10 , resnet18, Swish

resnet = ResNet18(10)
resnet.load_state_dict(torch.load('./checkpoint/ckpt_resnet18_Swish.pth')['net'])
print_act_p(resnet)

NameError: name 'ResNet18' is not defined

In [8]:
# cifar100 , resnet18, SiLU

resnet = ResNet18(100,act=SiLU())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_SiLU_199_76.74.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

76.74
199


In [12]:
# cifar100 , resnet18, SiLU

resnet = ResNet18(100,act=Swish())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_Swish_199_77.92.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([6.4133], requires_grad=True)
78.13
193


In [15]:
# cifar100 , resnet18, SwishT

resnet = ResNet18(10,act=SwishT())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_SwishT.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([-6.1995], requires_grad=True)
94.3
162


In [15]:
# cifar100 , resnet18, SwishT

resnet = ResNet18(100,act=Swish())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_gelu_199_76.87.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([5.5985], requires_grad=True)
77.09
198


In [4]:
# cifar100 , resnet18, SwishT

resnet = ResNet18(100,act=SwishT())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_sasn_199_79.59.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([6.4185], requires_grad=True)
79.68
194


In [3]:
# cifar100 , resnet18, SliuT

resnet = ResNet18(100,act=SliuT())
state_dict = torch.load('./checkpoint/ckpt_resnet18_cifar100_SliuT.pth')
resnet.load_state_dict(state_dict['net'])
print_act_p(resnet)
print(state_dict['acc'])
print(state_dict['epoch'])

Parameter containing:
tensor([-7.1654], requires_grad=True)
77.72
193


In [13]:
# cifar100 , swin, sASN

swin,_ = Swin_transformer(num_classes=100,image_size=32,act=sASN())
state_dict = torch.load('./checkpoint/ckpt_swin_sasn_199_65.91.pth')
swin.load_state_dict(state_dict['net'])
print_act_p(swin)
print(state_dict['acc'])
print(state_dict['epoch'])

n_classes: 100 img_size: 32 act: SwishT()


NameError: name 'sASN' is not defined

In [8]:
# cifar100 , swin, gelu

swin,_ = Swin_transformer(num_classes=100,image_size=32,act=nn.GELU())
state_dict = torch.load('./checkpoint/ckpt_swin_gelu_199_54.91.pth')
swin.load_state_dict(state_dict['net'])
print_act_p(swin)
print(state_dict['acc'])
print(state_dict['epoch'])

n_classes: 100 img_size: 32 act: GELU(approximate='none')
55.06
197


In [9]:
# cifar100 , swin, gelu

swin,_ = Swin_transformer(num_classes=100,image_size=32,act=sASN())
state_dict = torch.load('./checkpoint/ckpt_swin_sASN_199_56.20.pth')
swin.load_state_dict(state_dict['net'])
print_act_p(swin)
print(state_dict['acc'])
print(state_dict['epoch'])

n_classes: 100 img_size: 32 act: sASN()
Parameter containing:
tensor([1.0343], requires_grad=True)
56.3
198


In [4]:
swin,_ = Swin_transformer(num_classes=100,image_size=32,act=Swish())
state_dict = torch.load('./checkpoint/ckpt_swin_Swish_199_54.90.pth')
swin.load_state_dict(state_dict['net'])
print_act_p(swin)
print(state_dict['acc'])
print(state_dict['epoch'])

n_classes: 100 img_size: 32 act: Swish()
Parameter containing:
tensor([1.0149], requires_grad=True)
55.45
192


In [2]:
# cifar100 , swin, sASN

swin,_ = Swin_transformer(num_classes=100,image_size=32,act=SiLU())
state_dict = torch.load('./checkpoint/ckpt_swin_SiLU_199_55.30.pth')
swin.load_state_dict(state_dict['net'])
print_act_p(swin)
print(state_dict['acc'])
print(state_dict['epoch'])

n_classes: 100 img_size: 32 act: SiLU()
55.61
191


In [13]:
swin,_ = Swin_transformer(num_classes=100,image_size=32,act=SiLU())
state_dict = torch.load('./checkpoint/ckpt_swin_SiLU.pth')
swin.load_state_dict(state_dict['net'])
print_act_p(swin)
print(state_dict['acc'])
print(state_dict['epoch'])

n_classes: 100 img_size: 32 act: SiLU()
55.61
191
